In [1]:
import numpy as np
from functools import partial
from math import sin, cos, pi, sqrt
from tqdm import tqdm
import matplotlib.animation as animation
from matplotlib import rc
import matplotlib.pyplot as plt
import seaborn as sb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

rc('animation', html='jshtml')

In [2]:
class Waves:
    len = 1 # length AB=BC=CA
    N = 64 # number of knots for x
    h = len / N # mesh diameter for x

    t = 0.01 # time step
    n = 256 # number of time steps
    T = t * n # finish time

    def __init__(self, phi_A, phi_B, phi_C, u0, u1, n=256, t=0.01, N=64, len=1):
        self.len = len
        self.N = N
        self.h = len / N

        self.t = t
        self.n = n
        self.T = self.t * self.n

        self.phi_A = phi_A
        self.phi_B = phi_B
        self.phi_C = phi_C

        self.u0 = u0
        self.u1 = u1

    def compute_one(self, u, i, it, phi0_t, phiX_t):
        if (i == 0):
            return phi0_t(it * self.t)
        if (i == self.N):
            return phiX_t(it * self.t)
        d2x = (u[1, (i - 1)] - 2 * u[1, i] + u[1, (i + 1)]) / self.h**2
        return 2 * u[1, i] - u[0, i] + self.t**2 *  d2x

    def compute_task(self, phi0_t, phiX_t):
        res = []
        u = np.zeros((3, self.N+1))
        u[0, :] = np.fromfunction(self.u0, (self.N+1,))
        u[1, :] = np.fromfunction(self.u1, (self.N+1,))

        for it in range(self.n):
            for i in range(0, self.N + 1):
                u[2, i] = self.compute_one(u, i, it, phi0_t, phiX_t)
            res.append(list(u[2, :]))
            u[0, :] = u[1, :]
            u[1, :] = u[2, :]
            u[2, :] = 0

        return res

    def compute_results(self):
        BA_res = np.array(self.compute_task(self.phi_B, self.phi_A))
        AC_res = np.array(self.compute_task(self.phi_A, self.phi_C))
        CB_res = np.array(self.compute_task(self.phi_C, self.phi_B))

        AC_res = np.delete(AC_res, 0, 1)
        CB_res = np.delete(CB_res, 0, 1)

        return np.hstack((BA_res, AC_res, CB_res))

    def print_solution(self, u: np.array, y_lim=[-0.2, 0.2], x_beg=0, x_end=3):
        if len(u) == 0 or len(u[0]) == 0:
            print('Bad array')
            return 1

        x = [i * self.h for i in range(self.N + 1)] + \
            [self.len + i * self.h for i in range(1, self.N + 1)] + \
            [self.len + self.len + i * self.h for i in range(1, self.N + 1)]
        y = u[0]

        fig = plt.figure(figsize=(8,4))
        ax  = fig.add_subplot(111)
        ax.set_ylim(y_lim)

        ax.set_xlim([x_beg, x_end])
        ax.set_xticks(np.arange(x_beg, x_end + 1), labels=['B', 'A', 'C', 'B'])
        ax.grid(True)

        data, = ax.plot(x, y)
        def animate(i):
            y = u[i]
            data.set_data(x, y)
            return [data]

        return animation.FuncAnimation(fig, animate, frames=len(u), interval=500, blit=True)

    def print_solution_3d(self, u: np.array):
        x_BC = np.linspace(0, self.len / 2, self.N + 1)[::-1][1::]
        y_BC = -sqrt(3) * x_BC + sqrt(3) / 2

        x_BA = np.linspace(-self.len / 2, 0, self.N + 1)[::-1]
        y_BA = sqrt(3) * x_BA + sqrt(3) / 2

        x_AC = np.linspace(-self.len / 2, self.len / 2, self.N + 1)[1::]
        y_AC = np.zeros_like(x_AC)

        x = np.hstack((x_BA, x_AC, x_BC))
        y = np.hstack((y_BA, y_AC, y_BC))

        # Create figure
        fig = go.Figure()

        texts = [""] * x.shape[0]
        texts[0] = "B"
        texts[self.N] = "A"
        texts[2 * self.N] = "C"

        for step in np.arange(0, self.n):

            fig.add_trace(go.Scatter3d(
                visible=False,
                x=x,
                y=y,
                z=res[step],
                mode='lines+text',
                line=dict(width=4),
                text=texts
            ))


        steps = []
        for i in range(len(fig.data)):
            step = dict(
                method="update",
                args=[{"visible": [False] * len(fig.data)},
                    {"title": "Time: " + str(i * self.t)[:4]}],
                label=str(i * self.t)[:4] * (i % 2 == 0),
            )
            step["args"][0]["visible"][i] = True
            steps.append(step)

        sliders = [dict(
            active=0,
            currentvalue={"prefix": "Time: "},
            pad={"t": 50},
            steps=steps
        )]

        fig.update_layout(
            sliders=sliders
        )

        fig.show()


In [3]:
phi_A = lambda t: 0.04 * sin(t * 10)
phi_B = lambda t: 0
phi_C = lambda t: 0.04 * sin(t * 10 + pi)
u0 = lambda t: 0
u1 = lambda t: 0

wv = Waves(phi_A, phi_B, phi_C, u0, u1)
res = wv.compute_results()
wv.print_solution_3d(res)